In [198]:
import numpy as np
import random
import math

In [327]:
def get_distance2D(p1, p2):
    return math.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

def get_distance3D(p1, p2):
 
 return math.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2 + (p1[2] - p2[2]) ** 2)

#### Ground to Air Channel Gain

def get_PL_A2A(p1, p2):
 
 d_3D = get_distance3D(p1, p2)
 
 return 20 * math.log(d_3D) + 20 * math.log(f_c) + 20 * math.log((4*math.pi) / (3*10**8))

def get_PL_A2G(p1, p2):
 
#  d_3D = get_distance3D(p1, p2)
 eta_LOS, eta_NLOS = 2.0, 21.0 # https://ieeexplore.ieee.org/document/9844892
 PL_A2G_LOS = get_PL_A2A(p1, p2) + eta_LOS
 PL_A2G_NLOS = get_PL_A2A(p1, p2) + eta_NLOS

 return PL_A2G_LOS, PL_A2G_NLOS

def get_Pr_A2G(p1, p2):
 
 alpha = 0.136 # https://ieeexplore.ieee.org/document/8796414
 beta = 11.95 # https://ieeexplore.ieee.org/document/9844892
 d_3D = get_distance3D(p1, p2)
 elevation_angle = math.asin(p2[2]/d_3D)

 return 1 / (1 + alpha * math.exp(-1 * beta * (elevation_angle - alpha)))

def get_channel_gain_A2G(p1, p2):
 
 PL_A2G_LOS, PL_A2G_NLOS = get_PL_A2G(p1, p2)
 Pr_A2G = get_Pr_A2G(p1, p2)
 PL_A2G_dB = (Pr_A2G * PL_A2G_LOS) + ((1 - Pr_A2G) * PL_A2G_NLOS)
 PL_A2G = 10**(PL_A2G_dB / 10)

 return 1 / PL_A2G

#### Ground to Ground Channel Gain

def get_breakpoint(p1, p2):
    return 4 * (p1[2] - 1) * (p2[2] - 1) * f_c / (3*10**8)

def get_Pr_G2G(p1, p2):
    
    d_2D = get_distance2D(p1, p2)
    
    if d_2D < 18 or d_2D == 18:
        pr_LOS = 1
    else:
        pr_LOS = ((18/d_2D) + (math.exp(-d_2D/63)*(1-(18/d_2D))))
        
    return pr_LOS 

def get_PL_LOS_G2G(p1, p2):
    
    d_2D = get_distance2D(p1, p2)
    BP = get_breakpoint(p1, p2)
    
    if  (10 < d_2D) and (d_2D < BP):
        PL_LOS = 28 + 22*math.log10(get_distance3D(p1, p2)) + 20*math.log10(f_c*10**(-9))
    else:
        PL_LOS = 28 + 40*math.log10(get_distance3D(p1, p2)) + 20*math.log10((f_c*10**(-9))) - 9*math.log10(BP**2 + (p2[2] - p1[2])**2)
    
    return PL_LOS
def get_PL_NLOS_G2G(p1, p2):
    
    PL_NLOS = max(get_PL_LOS_G2G(p1, p2), (13.54 + 39.08 * math.log10(get_distance3D(p1, p2)) + 20 * math.log10(f_c*10**(-9)) - 0.6*(p1[2] -1.5)))
    
    return PL_NLOS

def get_PL_G2G(p1, p2):
    
    return (get_Pr_G2G(p1, p2) * get_PL_LOS_G2G(p1, p2)) + ((1-get_Pr_G2G(p1, p2)) * get_PL_NLOS_G2G(p1, p2))

def get_channel_gain_G2G(p1, p2):
    
    return 1 / 10**(get_PL_G2G(p1, p2)/10)

In [328]:
# Genrating increasing random numbers
def generate_increasing_random_numbers(n, start, end):
    # Generate n random numbers in the range and sort them
    random_numbers = sorted(random.sample(range(start, end), n))
    return random_numbers

In [329]:
n = 10  # Number of random numbers to generate
random_numbers = generate_increasing_random_numbers(n)
print(random_numbers)

TypeError: generate_increasing_random_numbers() missing 2 required positional arguments: 'start' and 'end'

In [488]:
#### Topology
f_c = 5 * 10 **9
#x_f_p = 0.3
P_max = 0.1
I0 = 3.98* 10 ** (-21)
W = 5 * 10 ** 6
responders = 12
x_f_p = np.linspace(0.1, 0.3, responders)
responder_height = 1.5
uavs = 3
alpha = 0.2
beta =0.2
loss_aversion = 5
base_station_coordinates = [0, 0, 25]
uav_coordinates = [[50, 0, 75], [60, 0, 75], [70, 0, 75]]
uav_coordinates.sort(reverse= True)
maximum_tolerable_interference_uav = [10, 20, 30]
w1 = 1
w2 = 100
#2D-List of responders having y, z coordinates 
responder_coordinates = [[0,responder_height] for x in range(responders)]

responder_uav_distance = list()
responder_bs_distance = list()

#Adding x coordinates to 2D-List of responders to make it 3-D coordinate list
for responder in range(responders):
    responder_coordinates[responder].insert(0, 80+(responder+1)+((responder+1)*20))
print(responder_coordinates)

for responder in range(responders):
    # Temporary List for Each responder which is a row matrix with 3 entries as 3 UAVs.
    responder_uav_distance_temp = list()
    for uav in range(uavs):
        responder_uav_distance_temp.append(get_distance3D(responder_coordinates[responder], uav_coordinates[uav]))
    responder_uav_distance.append(responder_uav_distance_temp)
print(responder_uav_distance)

for responder in range(responders):
    responder_bs_distance.append(get_distance3D(responder_coordinates[responder], base_station_coordinates))
print(responder_bs_distance)

[[101, 0, 1.5], [122, 0, 1.5], [143, 0, 1.5], [164, 0, 1.5], [185, 0, 1.5], [206, 0, 1.5], [227, 0, 1.5], [248, 0, 1.5], [269, 0, 1.5], [290, 0, 1.5], [311, 0, 1.5], [332, 0, 1.5]]
[[79.7699818227383, 84.16204607778973, 89.46088530749067], [90.03471552684553, 96.15742301039478, 102.8895038378551], [103.59174677550331, 110.86590999942227, 118.5379686007821], [119.32413837945782, 127.35089320456296, 135.64014892353958], [136.48168375280252, 145.00775841312768, 153.71158056568152], [154.5905883293029, 163.45718093739413, 172.44781819437438], [173.35296363200717, 182.45889948149966, 191.65398508770957], [192.57790631326327, 201.85700384182857, 211.20191760493086], [212.13969454112072, 221.5473989917282, 231.0048700785332], [231.9531202635567, 241.4585885819761, 251.00249002748956], [251.95882600139254, 261.54014988142836, 271.1517103025537], [272.11440608685166, 281.75565655368837, 291.4210870887692]]
[103.69787847395915, 124.2427060233316, 144.918080307462, 165.6751339217808, 186.48659469

In [489]:
print(x_f_p)

[ 0.1         0.11818182  0.13636364  0.15454545  0.17272727  0.19090909
  0.20909091  0.22727273  0.24545455  0.26363636  0.28181818  0.3       ]


In [490]:
responders

12

In [491]:
responder_uav_distance

[[79.7699818227383, 84.16204607778973, 89.46088530749067],
 [90.03471552684553, 96.15742301039478, 102.8895038378551],
 [103.59174677550331, 110.86590999942227, 118.5379686007821],
 [119.32413837945782, 127.35089320456296, 135.64014892353958],
 [136.48168375280252, 145.00775841312768, 153.71158056568152],
 [154.5905883293029, 163.45718093739413, 172.44781819437438],
 [173.35296363200717, 182.45889948149966, 191.65398508770957],
 [192.57790631326327, 201.85700384182857, 211.20191760493086],
 [212.13969454112072, 221.5473989917282, 231.0048700785332],
 [231.9531202635567, 241.4585885819761, 251.00249002748956],
 [251.95882600139254, 261.54014988142836, 271.1517103025537],
 [272.11440608685166, 281.75565655368837, 291.4210870887692]]

In [492]:
x_f_p[1]

0.11818181818181819

In [493]:
#Assignment List (Matching List)
responders_per_uav = [[] for x in range(uavs)]
# responders_per_uav = [[]*3] or [[]*uavs]
# for uav in range(uavs): responders_per_uav.append([]) 

def get_data_rate_res_uav(responder_id, uav_id):
    # Interference List
    responder_to_uav_channel_gain_power = list()
    #print(responders_per_uav[uav_id])
    
    # Compute the interference of other responders assigned to the same UAV
    for responder in responders_per_uav[uav_id]:
        if responder == responder_id:
            continue
        else:
            responder_to_uav_channel_gain_power.append(get_channel_gain_A2G(responder_coordinates[responder], uav_coordinates[uav_id]) *x_f_p[responder]*P_max)
    
    # Compute the interference of responders assigned to other UAVs
    responder_to_other_uav_channel_gain_power = list()
    for uav in range(uavs):
        if uav == uav_id:
            continue
        else:
            for responder in responders_per_uav[uav]:
                responder_to_other_uav_channel_gain_power.append(get_channel_gain_A2G(responder_coordinates[responder], uav_coordinates[uav]) *x_f_p[responder]*P_max)
    
    interference = sum(responder_to_other_uav_channel_gain_power) + sum(responder_to_uav_channel_gain_power)
    
    data_rate = W * math.log2(1 + ((x_f_p[responder_id] * P_max * get_channel_gain_A2G(responder_coordinates[responder_id], uav_coordinates[uav_id]))/(I0 + interference)))
    return data_rate

def get_data_rate_res_bs(responder_id):

    responder_to_bs_channel_gain_power = list()
    for responder in range(responders):
        if responder == responder_id:
            continue
        else:
            responder_to_bs_channel_gain_power.append(get_channel_gain_G2G(responder_coordinates[responder], base_station_coordinates) *(1 - x_f_p[responder])*P_max)
    data_rate = W * math.log2(1 + ((x_f_p[responder] * P_max * get_channel_gain_G2G(responder_coordinates[responder_id], base_station_coordinates))/(I0 + sum(responder_to_bs_channel_gain_power))))
    return data_rate

In [494]:
responders

12

In [495]:
# Set the data generated by responders (in KB)
data_per_responder = generate_increasing_random_numbers(responders, 1000, 5000)
data_per_responder

[1021, 1052, 1763, 1810, 2352, 2887, 3117, 3785, 4204, 4469, 4649, 4898]

In [496]:
get_channel_gain_A2G(responder_coordinates[0], uav_coordinates[0]), get_channel_gain_A2G(responder_coordinates[0], uav_coordinates[1]), get_channel_gain_A2G(responder_coordinates[0], uav_coordinates[2])

(2.2533826798861113e-20, 1.7605135849826264e-20, 1.3289786511849228e-20)

In [497]:
get_data_rate_res_uav(0, 0), get_data_rate_res_uav(0, 0), get_data_rate_res_uav(0, 0)

(397267.10767949343, 397267.10767949343, 397267.10767949343)

In [498]:
get_data_rate_res_bs(0)

2713809.7122027185

In [499]:
# Computing Latency and Energy overheads (Joules) to UAVs (sec)

data_offload_percentage_uav = 0.7

def get_latency_res_uav(responder_id, uav_id):
    # Convert the data transmitted to bits.
    responder_to_uav_latency = (data_offload_percentage_uav*data_per_responder[responder_id]*(10**3)*8)/get_data_rate_res_uav(responder_id, uav_id)
    return responder_to_uav_latency
    
    
def get_energy_res_uav(responder_id, uav_id):
    responder_to_uav_energy = get_latency_res_uav(responder_id, uav_id)*(x_f_p[responder_id]*P_max)
    return responder_to_uav_energy


In [500]:
get_latency_res_uav(0, 0), get_energy_res_uav(0, 0)

(14.392331732162523, 0.14392331732162525)

In [501]:
# Computing Latency and Energy overheads (Joules) to Base station (sec)

data_offload_percentage_bs = 0.3

def get_latency_res_to_bs(responder_id):
    responder_to_bs_latency = (data_offload_percentage_bs*data_per_responder[responder_id]*(10**3)*8)/get_data_rate_res_bs(responder_id)
    return responder_to_bs_latency

def get_energy_res_bs(responder_id):
    responder_to_bs_energy = get_latency_res_to_bs(responder_id)*((1 - x_f_p[responder_id])*P_max)
    return responder_to_bs_energy


In [502]:
get_latency_res_to_bs(0), get_energy_res_bs(0)

(0.9029372947490424, 0.081264356527413831)

In [503]:
# Computing the total overhead (Latency) and (Energy) for a specific responder to a UAV

def get_total_latency_res(responder_id, uav_id):
    total_latency_res =  get_latency_res_uav(responder_id, uav_id) + get_latency_res_to_bs(responder_id)
    return total_latency_res

def get_total_energy_res(responder_id, uav_id):
    total_energy_res = get_energy_res_uav(responder_id, uav_id) + get_energy_res_bs(responder_id)
    return total_energy_res

In [504]:
get_total_latency_res(0, 0), get_total_energy_res(0, 0)

(15.295269026911566, 0.22518767384903909)

In [505]:
# Get the min latency profile corresponding to a responder.
def get_latency_profile_res(responder_id):
    latency_profile = list()
    for uav in range(uavs):
        latency_profile.append(get_total_latency_res(responder_id, uav))
    return latency_profile

# Get the min energy corresponding to a responder.
def get_energy_profile_res(responder_id):
    energy_profile = list()
    for uav in range(uavs):
        energy_profile.append(get_total_energy_res(responder_id, uav))
    return energy_profile

In [506]:
get_latency_profile_res(1), get_energy_profile_res(1)

([25.603006509238156, 33.147694530187046, 43.704116655612815],
 [0.59912727085631656, 0.6882917656493488, 0.81304948167710789])

In [507]:
# Construct the set of Latency and Energy Profiles of all responders in a 2D Matrix
global_latency_profile = list()
global_energy_profile = list()

for responder in range(responders):
    global_latency_profile.append(get_latency_profile_res(responder))
    global_energy_profile.append(get_energy_profile_res(responder))
print(global_latency_profile)
print(global_energy_profile)

[[15.295269026911566, 19.215357070311658, 25.034544205515175], [25.603006509238156, 33.147694530187046, 43.704116655612815], [76.22628685513001, 97.915772855088, 127.1067741370205], [138.66349770530286, 174.66317600274763, 221.60165070904748], [307.8615484391058, 379.83628151269136, 471.2562277939758], [617.7601789864445, 748.215055806977, 910.4016958203664], [1045.193794095006, 1246.2913329767252, 1491.9438081133985], [1915.728700942651, 2254.8815329479185, 2663.1518396863066], [3109.6406248550243, 3620.9047416979606, 4228.752934836468], [4699.296768110382, 5422.5892751544125, 6273.300790437532], [6786.63137238345, 7770.97322146914, 8917.764337912446], [9724.623981558723, 11060.844341003267, 12604.391739989565]]
[[0.22518767384903909, 0.26438855428304003, 0.32258042563507522], [0.59912727085631656, 0.6882917656493488, 0.81304948167710789], [2.23778690519045, 2.533552623371695, 2.9316117317616834], [4.580521379396723, 5.1368800439935969, 5.8622928349091401], [10.872771750650248, 12.115

In [508]:
# Get the total Overhead of a responder to a UAV [Unitless Quantity]
def get_total_overhead_res_to_uav(responder_id, uav_id):
    total_overhead = w1*get_total_latency_res(responder_id, uav_id) +w2*get_total_energy_res(responder_id, uav_id)
    return total_overhead

In [509]:
get_total_overhead_res_to_uav(0, 0)

37.814036411815479

In [532]:
# Compute the Probability of Failure (PoF) of a responder with a specific UAV (Prospect Theory)
responder_to_uav_channel_gain_power = list()
responder_to_other_uav_channel_gain_power = list()
interference = 1e-10
def get_prob_failure_uav(responder_id, uav_id):
    # Compute interference with responders assigned to same UAV
    for responder in responders_per_uav[uav_id]:
        if responder == responder_id:
            continue
        else:
            responder_to_uav_channel_gain_power.append(get_channel_gain_A2G(responder_coordinates[responder], uav_coordinates[uav_id]) *x_f_p[responder]*P_max)
    
    # Compute the interference of responders assigned to other UAVs
    for uav in range(uavs):
        if uav == uav_id:
            continue
        else:
            for responder in responders_per_uav[uav]:
                responder_to_other_uav_channel_gain_power.append(get_channel_gain_A2G(responder_coordinates[responder], uav_coordinates[uav]) *x_f_p*P_max)
                
    interference = sum(responder_to_other_uav_channel_gain_power) + sum(responder_to_uav_channel_gain_power)
    print(interference)
    
    if interference == 0:
        PoF_res_uav = 0
    elif interference < maximum_tolerable_interference_uav[uav_id]:
        PoF_res_uav = maximum_tolerable_interference_uav[uav_id]/interference
    else:
        PoF_res_uav = 1
        
    return PoF_res_uav

In [511]:
prob_failure_res_uav(0, 0), data_per_responder[0]

0


(0, 1021)

In [512]:
# Compute the Expected Overhead
def get_expected_overhead_res_to_uav(responder_id, uav_id):
    uav_failed_overhead = w1*get_latency_res_uav(responder_id, uav_id) + w2*get_energy_res_uav(responder_id, uav_id)
    sending_to_bs = w1*((data_per_responder[responder_id]*(10**3)*8)/get_data_rate_res_bs(responder_id)) + w2*(((data_per_responder[responder_id]*(10**3)*8)/get_data_rate_res_bs(responder_id))*((1-x_f_p[responder_id])*P_max))
    first_term = get_prob_failure_uav(uav_id)*(uav_failed_overhead + sending_to_bs)
    second_term = (1-get_prob_failure_uav(uav_id))*(get_total_overhead_res_to_uav(responder_id, uav_id))
    overhead = first_term + second_term
    return overhead

In [513]:
get_expected_overhead_res_to_uav(2, 2)

0
0


420.26794731318887

In [514]:
def get_the_uav_overhead(responder_id, uav_id):
    overhead = w1*get_latency_res_uav(responder_id, uav_id) + w2*get_energy_res_uav(responder_id, uav_id)
    return overhead

In [527]:
get_the_uav_overhead(0, 0)

28.78466346432505

In [516]:
def get_the_reference_point_res_to_uav(responder_id, uav_id):
    term1 = (((data_offload_percentage_uav*data_per_responder[responder_id])*(10**3)*8)/get_data_rate_res_bs(responder_id))
    term2 = term1 * ((1-x_f_p[responder_id])*P_max)
    reference_point = w1*term1 + w2*term2
    return reference_point

In [528]:
get_the_reference_point_res_to_uav(0, 0)

21.068536877477658

In [522]:
def get_prospect_theoretic_utility_res_to_uav(responder_id, uav_id):
    if get_the_uav_overhead(responder_id, uav_id) <= get_the_reference_point_res_to_uav(responder_id, uav_id):
        prospect_thoeretic_utility = (get_the_reference_point_res_to_uav(responder_id, uav_id) - get_the_uav_overhead(responder_id, uav_id))**alpha
    else:
        prospect_thoeretic_utility = -loss_aversion *((get_the_uav_overhead(responder_id, uav_id) - get_the_reference_point_res_to_uav(responder_id, uav_id))**beta)
    return prospect_thoeretic_utility

In [529]:
get_prospect_theoretic_utility_res_to_uav(0, 0)

-7.5240188185195001

In [ ]:
def get_expected_prospect_theoretic_utility_res_to_uav(responder_id, uav_id):
    expected_overhead = (1-get_prob_failure_uav(uav_id))